In [52]:
import numpy as np
import pandas as pd
#import shapefile as shp
import matplotlib.pyplot as pplot
import seaborn as sns
import geopandas
from shapely.geometry import Point
import json

## Figuring out the transportation graph between 31 major Greek cities:
### Methodology 
This model assumes that transmission from closed community to closed community is occurring mostly due to mass transportation media. These are busses, airplanes and ships. For cities in southern mainland Greece, mostly cities close to Athens, train is a very common means of transportation.  

The first case of COVID19 in Greece was in Thessaloniki, while a second one was a mass contagion case from pilgrims returning from Israel.

Most of mass transportation in Greece happens with buses and ships. Flights are common between Athens, Thessaloniki and some major islands such as Creta and Rhodes. This is where the first most important out-breaks are expected since these cities are also the most densely populated and have airports. 

The frequency of public transportation means between the 31 major Greek cities is used as a surrogate to the real population flow from city to city. I need the assistance of transportation engineers to make this model realistic. 


## Approximation of edge strengths
* As a first approximation on the edge strength I model manually the population flow between airports as 1000 individuals per day, which are all infected if an infected individual is on the same flight.
* Next, the population flow between other cities is modeled as proportional to a factor that is the sum of the population of the cities over the population of all the cities, divided by the third power of their distance in km. This is to model the common effect of people working in different parts than they live.

In [53]:
import pandas as pd
cities = pd.read_csv("greek_cities.tsv", sep = "\t")
#periphereies = geopandas.read_file("periphereies_wsg/d7f50467-e5ef-49ac-a7ce-15df3e2ed738.dbf", encoding = "utf-8")


In [54]:
latlon = cities['Latitude (DD)']

In [55]:
# Compute approximate population movement strengths:
population = cities.Pop2000.values;
names = cities.index;
v = cities[['Latitude (DD)','Longitude (DD)']].values

edge_entries = [];
for k1,v_1 in enumerate(v):
    for k2,v_2 in enumerate(v):
        if k1 is not k2:
            val = (population[k1] + population[k2])*0.01/np.sum((5+abs(v_1 - v_2))**3)
            if val < 5:
                continue
            edge_entries.append((int(cities.iloc[k1].name[0]),int(cities.iloc[k2].name[0]),val))
            #print((names[k1],names[k2],val))
            
            


array([762100, 372100, 179600, 164000, 141000, 133800, 127200, 112100,
        95200,  89200,  84300,  82700,  82200,  77800,  77800,  77600,
        77000,  75700,  75600,  73300,  71000,  69000,  67500,  66600,
        66200,  63000,  62500,  60000,  59300,  58100,  55500,  54800,
        54600,  53800,  53500,  51600,  50300,  50000,  49800,  49400,
        47400,  47000,  46200,  46100,  44900,  44300,  43700,  43400,
        43300,  42700,  42600,  41900,  38900,  37000,  36000,  33900,
        33700,  31200,  31000,  30500,  29300,  28800,  28500,  28400,
        27900,  27000,  27000,  26900,  26900,  26400,  26300,  26300,
        26200,  26100,  26100,  24800,  24600,  24300,  24200,  23800,
        23700,  23500,  23300,  22700,  22100,  21700,  21500,  20500,
        20300,  20300,  20000,  19900,  18700,  18700,  18300,  17800,
        17800,  17100,  16700,  16200,  15800,  15600,  15400,  15300,
        15200,  15100,  15100,  14900,  14600,  14300,  14200,  14100,
      

In [60]:
links = [{"source" : e1, "target" : e2, "count" : vv } for e1,e2,vv in edge_entries]
nodes = [{ "id" : v1[0],"name" : v1[1] , "city" : v1[1], "faa" : v1[1].upper()[0:3],"lat" : v2[0], "lon" : v2[1] ,"population" : int(p) } for v1,v2,p in zip(names.to_numpy(),v, population)]
data = {"nodes" : nodes, "links" : links }

with open("data/greek_graph.json", "w") as f:
    f.write(json.dumps(data))

## Plot Greece and connections between cities:

In [51]:
with open("data/data.json",'r') as f:
    d = f.read()
dd = json.loads(d)
with open("data/data_alt.json","w") as r:
    r.write(json.dumps(dd))

{'nodes': [{'id': 3448,
   'name': 'General Edward Lawrence Logan Intl',
   'city': 'Boston',
   'faa': 'BOS',
   'lat': 42.364347,
   'lon': -71.005181},
  {'id': 3453,
   'name': 'Metropolitan Oakland Intl',
   'city': 'Oakland',
   'faa': 'OAK',
   'lat': 37.721278,
   'lon': -122.220722},
  {'id': 3454,
   'name': 'Eppley Afld',
   'city': 'Omaha',
   'faa': 'OMA',
   'lat': 41.303167,
   'lon': -95.894069},
  {'id': 3457,
   'name': 'Wichita Mid Continent',
   'city': 'Wichita',
   'faa': 'ICT',
   'lat': 37.649944,
   'lon': -97.433056},
  {'id': 3458,
   'name': 'Kansas City Intl',
   'city': 'Kansas City',
   'faa': 'MCI',
   'lat': 39.297606,
   'lon': -94.713905},
  {'id': 3459,
   'name': 'Dane Co Rgnl Truax Fld',
   'city': 'Madison',
   'faa': 'MSN',
   'lat': 43.139858,
   'lon': -89.337514},
  {'id': 3462,
   'name': 'Phoenix Sky Harbor Intl',
   'city': 'Phoenix',
   'faa': 'PHX',
   'lat': 33.434278,
   'lon': -112.011583},
  {'id': 3467,
   'name': 'Spokane Intl',
   

In [22]:
#ax = periphereies.plot(figsize = (20,20))
ax = pplot#.figure()
ax.scatter(latlon[:, 0] , latlon[:,1], c = 'r')


line_segments = [((cities.iloc[k1]['Longitude (DD)'],cities.iloc[k2]['Longitude (DD)']),(cities.iloc[k1]['Latitude (DD)'],cities.iloc[k2]['Latitude (DD)']),{"width" : v}) for k1,k2, v in edge_entries ]
for ls in line_segments:
    ax.plot(*ls[0:2],'.-C3',linewidth = ls[-1]['width']**0.1, alpha = 0.2)

    

NameError: name 'latlon' is not defined